## Assignment 1 Model Fitting with Maximum Likelihood 
* The purpose of this assignment is to learn how to write a log likelihood function, fit models to data with maximum likelihood and to select among models using AIC (or BIC)
* Along the way, we'll learn a bit about decision making models.  

# Due: Friday 10/17

In [5]:
import numpy as np
from numpy import random
from matplotlib import pyplot as plt

In [ ]:
def shiftedwald(params, x):
    '''
    params: is a list or numpy array containing two parameters
    x: are the data
    '''
 
    gamma = params[0] #drift rate
    alpha = params[1] #boundary separation
    theta = params[2] #shift or nondecision time 
    x = x-theta
    z = alpha/np.sqrt(2*np.pi*(x**3))
    w = ((np.abs(alpha-gamma*x))**2)/(2*x)
    f = z*np.exp(-w)
    return f

## Problem 1 
### Use the shifted Wald distribution defined above.  Write a function which computes the negative log likelihood 
### of the shifted Wald distribution.  Assume that the data to be analyzed is in a variable called data. 
### The function should return negative log likelihood 

In [ ]:
def negloglikeWald(params):
    ''' written with params containing gamma, alpha, theta in that order.  Assumes data is in the variable data'''
    return negloglike 

## Problem 2 
### Using the function the negative log-likelihood that you developed in problem 1, fit the shifted Wald distribution to the data for ALL the subjects in ReactionTimeData.csv.  
### You should consider two models: 
### Model A: All of the data comes from a single distribution. 
### Model B: The data in each condition (Easy, Medium, Hard) comes from a separate distribution. 

### When I did this, I chose to use the function `least_squares` in the scipy.optimize module, and my call looked like this. 
    data = rt([correct == 1])
    paramfit = least_squares(negloglikeWald,[0.1,80,250],bounds = ([0,0,0],np.inf),max_nfev=50000)
### I highly recommend that you investigate what least_squares does, its a super useful function.  
### If you run into problems with your computer taking too long, reduce the amount of data, even do just 1 or a few subjects.
### Remember, this model is only valid for correct responses. 
### Make a plot or table or something to show me (make it pretty!) how the resulting parameter fits turned out.  

## Problem 3 
### Use AIC to evaluate which model you should prefer.  Are all the data from 1 distribution or from three different distributions?  

## Problem 4 
### The two models given above are not the only models you could propose.  
### The original idea of this experiment was to manipulate drift rate in order to find brain activity related to speed of information processing. 
### So, our hypothesis was that non-decision time, and boundary would be the same for all conditions, and only the drift rate would vary.  
### Write a function for negative log likelihood for a model that keeps boundary (alpha) and non-decision time (theta) the same for all 3 conditions, but allows drift rate (gamma) to vary. 
### Hint: In this model there are now 5 parameters - gamma_easy, gamma_medium, gamma_hard, alpha, theta. 
### Hint: The likelihood should be evaluated using the correct gamma for each condition, and using the sama alpha and theta for all condition.  